In [1]:
# GPU setting
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# modules setting
from sklearn.metrics import confusion_matrix
import itertools
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import random
import cv2

/home/project/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# CORE DATA DIR

In [13]:
# cls = ['negative', 'hemorrhagic', 'depressed', 'protruded']
# lesion = {'negative' : ['negative'], 
#           'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding'],
#           'depressed': ['erosion', 'ulcer', 'stricture'],
#           'protruded': ['ampulla_of_vater', 'lymphoid_follicles', 'small_bowel_tumor']}
cls = ['negative', 'depressed']
lesion = {'negative' : ['negative'], 
          'depressed': ['erosion', 'ulcer', 'stricture']}

In [14]:
# path of data folder
dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/total/'
path_folder = {}

for i in cls:
    path_folder[i] = {}
    for j in lesion[i]:
        temp = dir_data + '{0}/{1}'.format(i, j)
        path_folder[i][j] = temp
        
print(path_folder)

{'negative': {'negative': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/total/negative/negative'}, 'depressed': {'erosion': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/total/depressed/erosion', 'ulcer': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/total/depressed/ulcer', 'stricture': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/total/depressed/stricture'}}


In [15]:
list_data_aug = {}
for i in cls:
    list_data_aug[i] = {}
    for j in lesion[i]:
        list_data_aug[i][j] = os.listdir(path_folder[i][j])

print(list_data_aug)

{'negative': {'negative': ['명나___02-27-38___2024017.jpg', 'JH___04-10-09___1030024.jpg', '동신___04-04-37___2044829.jpg', 'YK___03-50-31___1027669.jpg', 'JK___04-24-55___1031798.jpg', '희박___05-12-35___1037514.jpg', '인이___04-15-11___2055892.jpg', 'PM___02-36-20___2028425.jpg', 'mK___02-17-25___2029785.jpg', '왕신___04-28-48___2048286.jpg', '장최___03-32-53___2042616.jpg', '이동___04-16-26___2038107.jpg', '명나___02-27-24___2023979.jpg', 'JL___04-10-42___1030029.jpg', '용김___01-45-53___2024079.jpg', '광배___02-22-28___2031286.jpg', '창김___02-28-58___2034968.jpg', '화김___02-42-48___2028375.jpg', '보김___02-38-00___2029924.jpg', '민김___01-21-03___2021714.jpg', '동김___01-27-31___2021570.jpg', '기강___02-43-24___2030014.jpg', '기강___02-43-17___2029987.jpg', 'YK___04-44-07___1034102.jpg', '용김___01-44-55___2023901.jpg', '차박___01-00-00___1007200.jpg', '민김___01-31-26___2023131.jpg', 'HS___01-35-30___1011463.jpg', 'SL___03-38-13___1026192.jpg', '정주___10-28-04___2092504.jpg', '보김___01-08-28___2016250.jpg', 'YB___01-34-

In [17]:
# dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm/total/'

# data_x = {'negative':{'negative': []},
#          'hemorrhagic':{'red_spot':[],
#                        'angioectasia':[],
#                        'active_bleeding':[]},
#          'depressed':{'erosion':[],
#                      'ulcer':[],
#                      'stricture':[]},
#          'protruded':{'ampulla_of_vater':[],
#                      'lymphoid_follicles':[],
#                      'small_bowel_tumor':[]}}
data_x = {'negative':{'negative': []},
         'depressed':{'erosion':[],
                      'ulcer':[],
                      'stricture':[]}}

for i in cls:
    for j in lesion[i]:
        for k in list_data_aug[i][j]:
            temp = cv2.imread(dir_data + i + '/' + j + '/' + k)
            data_x[i][j].append([k, temp])

In [18]:
def sep(data_x):
    num_data = len(data_x)
    num_train_data = int(num_data*0.8)

    train_idx = set(np.random.choice(num_data, num_train_data, replace=False))
    test_idx = set(np.arange(len(data_x))) - train_idx
        
    train_x = [data_x[i] for i in train_idx]
    test_x = [data_x[i] for i in test_idx]
        
    return (train_x, test_x)

In [19]:
# train_data_x = {'negative':{'negative': []},
#                 'hemorrhagic':{'red_spot':[],
#                                'angioectasia':[],
#                                'active_bleeding':[]},
#                 'depressed':{'erosion':[],
#                              'ulcer':[],
#                              'stricture':[]},
#                 'protruded':{'ampulla_of_vater':[],
#                              'lymphoid_follicles':[],
#                              'small_bowel_tumor':[]}}
# test_data_x = {'negative':{'negative': []},
#                 'hemorrhagic':{'red_spot':[],
#                                'angioectasia':[],
#                                'active_bleeding':[]},
#                 'depressed':{'erosion':[],
#                              'ulcer':[],
#                              'stricture':[]},
#                 'protruded':{'ampulla_of_vater':[],
#                              'lymphoid_follicles':[],
#                              'small_bowel_tumor':[]}}
train_data_x = {'negative':{'negative': []},
                'depressed':{'erosion':[],
                             'ulcer':[],
                             'stricture':[]}}
test_data_x = {'negative':{'negative': []},
               'depressed':{'erosion':[],
                            'ulcer':[],
                            'stricture':[]}}

for i in cls:
    for j in lesion[i]:
        train_data_x[i][j], test_data_x[i][j] = sep(data_x[i][j])

In [20]:
train_dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/train/'

for i in cls:
    for j in lesion[i]:
        for k in train_data_x[i][j]:
            cv2.imwrite(train_dir_data + '{0}/{1}/{2}'.format(i,j,k[0]), k[1])

In [21]:
test_dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/nd/test/'

for i in cls:
    for j in lesion[i]:
        for k in test_data_x[i][j]:
            cv2.imwrite(test_dir_data + '{0}/{1}/{2}'.format(i,j,k[0]), k[1])